In [1]:
import pickle
import pandas as pd
import numpy as np
import time 
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

def feat_eng(df_fe):
    '''
    Función que realiza la selección de los features que serán utilizdos para la clasificación
    
    inputs: Data Frame limpio (df_clean.pkl)
    outputs: Data Frame con la matriz de diseño para el modelo (df_clean.pkl)
        
    '''
     
    # Transformación a OHE
    df_fe = df_fe.sort_values(by='inspection_date', ascending=True)
    df_input = pd.DataFrame(df_fe[['label_risk','label_results','zip','facility_type']])
    data_input_ohe = pd.get_dummies(df_input)
    etiqueta = data_input_ohe.label_results
    data_input_ohe= data_input_ohe.drop('label_results', axis = 1)
    variables_lista = list(data_input_ohe.columns)
    # Grid Search
    np.random.seed(20201124)
    # ocuparemos un RF
    classifier = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=1234)
    # separando en train, test
    #X_train, X_test, y_train, y_test = train_test_split(data_input_ohe, etiqueta, test_size=0.3)

    # definicion de los hiperparametros que queremos probar
    hyper_param_grid = {'n_estimators': [300, 400], #'min_samples_leaf': [3,5,7],
                        'max_depth': [7, 10],
                        'min_samples_split': [3],
                        'max_features': [10, 15, 20],
                        'criterion': ['gini']}
    # usamos TimeSeriesSplit para dividir respetando el orden cronológico
    tscv = TimeSeriesSplit(n_splits=3)
    # This was the trickiest part as a newbie. Straight from the docs
    # If you only have experience with CV splits this way
    # of making the splits might seem foreign. Fret not.
    for train_index, test_index in tscv.split(data_input_ohe):
        X_train, X_test = data_input_ohe.iloc[train_index, :], data_input_ohe.iloc[test_index,:]
        y_train, y_test = etiqueta.iloc[train_index], etiqueta.iloc[test_index]
    # ocupemos grid search
    gs = GridSearchCV(classifier, 
                           hyper_param_grid, 
                           scoring = 'precision', return_train_score=True,
                           cv = tscv)
    start_time = time.time()
    gs.fit(X_train, y_train)
    best_rf = gs.best_estimator_
    best_score = gs.best_estimator_.oob_score_
    feature_importance = pd.DataFrame({'importance':\
                                       best_rf.feature_importances_,\
                                       'feature': variables_lista})
    feature_importance=feature_importance.sort_values(by="importance", ascending=False)
    #fi_out = feature_importance.head(10)
    time_exec = time.time() - start_time
    nrows_ohe = data_input_ohe.shape[0]
    ncols_ohe = data_input_ohe.shape[1]
    #print("Tiempo en ejecutar: ", time.time() - start_time)
    return df_input, nrows_ohe, ncols_ohe, float(best_score), time_exec, str(best_rf)

In [2]:
df = pd.DataFrame(pickle.load(open("clean.pkl","rb")))

In [4]:
df_input, nrows_ohe, ncols_ohe, best_score, time_exec, best_rf = feat_eng(df)

In [5]:
df_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 0 to 609
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   label_risk     610 non-null    int64 
 1   label_results  610 non-null    int64 
 2   zip            610 non-null    object
 3   facility_type  610 non-null    object
dtypes: int64(2), object(2)
memory usage: 23.8+ KB


In [8]:
type(time_exec)

float

In [ ]:
columns = [("exec_date","date"),("exec_param","json"),("executer","varchar(10)"),
           ("source_path","text"),("nrow_ohe","int"),("ncols_ohe","int"),("best_score","real"),
           ("time_exec","real"),("best_rf","text")]